# Code to START STREAM

In [30]:
import requests

# Endpoint URL
url = "http://localhost:8000/start-synthetic-data"

# Payload
payload = {
    "num_players": 10,  # Replace with the desired number of players
    "stream_seconds": 50  # Replace with the desired stream duration
}

# Sending POST Request
response = requests.post(url, params=payload)

# Print Response
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Error:", response.status_code, response.text)


Success: {'message': 'Synthetic data stream started.'}


# Code to get AVERAGE SPEED / ACCELERATION

In [31]:
# Client code
import requests

# Base URL for the FastAPI app
base_url = "http://localhost:8000"
num_players = 3  # Number of players
run_id = "run_001"  # Run ID

# Construct the URL with path parameters
url = f"{base_url}/get-team-average-stats/{num_players}/{run_id}"

# Make the GET request
response = requests.get(url)

# Process the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code, response.text)

{'run_id': 'run_001', 'team_average_speed': 2.171549669012153, 'team_average_acceleration': -0.73626435100348}


# Code to get AVERAGE TEMPERATURE / HEART Rate

In [32]:
import requests

# API endpoint configuration
BASE_URL = "http://127.0.0.1:8000"
ENDPOINT = "/get-team-kafka-stats/{num_players}/{run_id}"

# Parameters
num_players = 3
run_id = "run_001"

# Build the full URL
url = BASE_URL + ENDPOINT.format(num_players=num_players, run_id=run_id)

try:
    # Make the GET request
    response = requests.get(url)

    # Check the response status code
    if response.status_code == 200:
        data = response.json()
        print(data)  # Print the response JSON
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Error: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


{'run_id': 'run_001', 'team_average_heart_rate': 118.0, 'team_average_temperature': 37.96666666666666}


# Code to get ALARM for HEART RATE

In [33]:
import requests

# API endpoint configuration
BASE_URL = "http://127.0.0.1:8000"
ENDPOINT = "/analyze-heart-rate/{num_players}/{run_id}"

# Parameters
num_players = 3
run_id = "run_001"

# Build the full URL
url = BASE_URL + ENDPOINT.format(num_players=num_players, run_id=run_id)

try:
    # Make the GET request
    response = requests.get(url)

    # Check the response status code
    if response.status_code == 200:
        data = response.json()
        print(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Error: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


{'run_id': 'run_001', 'analyzed_data': [{'device_id': 'player_heart_rate_1', 'heart_rate': 102, 'alarm_type': 'normal', 'timestamp': '2024-12-12T18:05:29.771678'}, {'device_id': 'player_heart_rate_2', 'heart_rate': 80, 'alarm_type': 'normal', 'timestamp': '2024-12-12T18:05:29.822486'}, {'device_id': 'player_heart_rate_3', 'heart_rate': 125, 'alarm_type': 'normal', 'timestamp': '2024-12-12T18:05:29.983439'}]}


# Code to get INSTANT GPS LOCATION

In [34]:
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the number of players and the run you want to check
num_players = 3  # Example: get GPS data for players 1 through 3
run = "run_001"
limit = 1  # Fetch the latest messages with this limit
base_url = "http://127.0.0.1:8000/get-topic-messages/{device_id}/{run_id}?limit={limit}"

# Function to make the request and process the response
def fetch_kafka_messages(device_id, run_id, limit):
    url = base_url.format(device_id=device_id, run_id=run_id, limit=limit)
    try:
        # Make the request to the FastAPI endpoint
        response = requests.get(url)
        # Check the status of the response
        if response.status_code == 200:
            response_json = response.json()
            return response_json  # Return the entire response JSON
        else:
            print(f"Error for {device_id}/{run_id}: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"An error occurred for {device_id}/{run_id}: {e}")
        return None

# Main function to run the requests concurrently
def main():
    # Generate the device ids based on the number of players
    device_ids = [f"gps_{i}" for i in range(1, num_players + 1)]

    # Use ThreadPoolExecutor to run requests concurrently
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Submit tasks to the executor for each player
        futures = {executor.submit(fetch_kafka_messages, device_id, run, limit): device_id for device_id in device_ids}
        
        for future in as_completed(futures):
            device_id = futures[future]
            try:
                response_data = future.result()
                if response_data:
                    print(json.dumps(response_data, indent=4))
                else:
                    print(f"No data returned for {device_id}/{run}.")
            except Exception as exc:
                print(f"An error occurred for {device_id}/{run}: {exc}")

# Run the main function
if __name__ == "__main__":
    main()


{
    "device_id": "gps_1",
    "run_id": "run_001",
    "messages": [
        {
            "latitude": 40.000462,
            "longitude": -75.000545,
            "timestamp": "2024-12-12T18:05:33.288986"
        }
    ]
}
{
    "device_id": "gps_2",
    "run_id": "run_001",
    "messages": [
        {
            "latitude": 40.000447,
            "longitude": -75.000547,
            "timestamp": "2024-12-12T18:05:33.300193"
        }
    ]
}
{
    "device_id": "gps_3",
    "run_id": "run_001",
    "messages": [
        {
            "latitude": 40.000515,
            "longitude": -75.00053,
            "timestamp": "2024-12-12T18:05:33.301820"
        }
    ]
}
